In [ ]:
#Sample dataset
training_data = [
    ["Green",3,"Mango"],
    ["Yellow",3,"Mango"],
    ["Red",1,"Grape"],
    ["Red",1,"Grape"],
    ["Yellow",3, "Lemon"]
]

#Column labels
header = ["color","diameter", "label"]

def unique_vals(rows,col):
    #Find unique vals for any col
    return set([row[col] for row in rows])

#unique_vals(training_data, 0)

def is_numeric(val):
    retrun isinstance(val, int) or isinstance(val, float)
    
#is_numeric(3)
#is_numeric("Red")

class Question:
    
    #Partition the dataset
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        #Compares feature value in one example to feature value in Question
        val = example[self.column]
        if is_numeric(val):
            return val>=self.value
        else:
            return val==self.value
        
    def __repr__(self):
        #Print question in readable format
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value ))
    
def partition(rows, question):
    #Partition dataset (For each row check if it matches the question)

    true_rows, false_rows = [],[]
    for row in rows:
        if(question.match(row)):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows        
        
#Let's partition data based on whether rows are Red or not
#partition(training_data, Question(0,' Red'))

def gini(rows):
    #Calc Gini Impurity for a list of rows
    counts = class_counts(rows)
    impurity = 1
    for lb1 in counts:
        prob_of_lb1 = counts[lb1]/float(len(rows))
        impurity - = prob_of_lb1**2
    return impurity

def info_gain(left, right, current_uncertainity):
    #Info Gain (Uncertainity of starting node minus weighted impurity of two child nodes)
    p = float(len(left))/(len(left)+len(right))
    return current_uncertainity - p * gini(left) - (1-p)*gini(right)

def find_best_split(rows):
    best_gain = 0  #keep track of the best information gain
    best_question = None  #keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns
    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in the column
        for val in values:  # for each value
            question = Question(col, val)
            #try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            #Skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            #Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            #You actually can use '>' instead of '>=' herebut I wanted the tree to look a certain way for our toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question
    
    #######
    # Demo:
    #Find the best question to ask first for our toy dataset.
    best_gain, best_question = find_best_split(training_data)
    best_question
    ##output - Is diameter >= 3?
    
def build_tree(rows):
    #Try partitioning the dataset on each of the unique attribute,
    #calculate the information gain,
    #and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
    #Base case: no further info gain
    #Since we can ask no further questions,
    #we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    #If we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)
    #Recursively build the true branch.
    true_branch = build_tree(true_rows)
    #Recursively build the false branch.
    false_branch = build_tree(false_rows)
    #Return a Question node.
    #This records the best feature / value to ask at this point, as well as the branches to follow depending on the answer.
    return Decision_Node(question, true_branch, false_branch)    